In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def create_fis_datasets(source_file='academicPerformanceData.xlsx'):
    print("⚙️ FIS İçin Veri Setleri Oluşturuluyor (Her sınıftan 20 örnek)...")
    
    if not os.path.exists(source_file):
        print(f"❌ HATA: '{source_file}' bulunamadı! Lütfen dosyayı ana dizine yükleyin.")
        return

    # 1. Excel Dosyasını Oku
    try:
        # Başlık bazen 2. satırda olabiliyor, header=1 deniyoruz
        df = pd.read_excel(source_file, header=1)
    except:
        df = pd.read_excel(source_file)

    # 2. Temizlik İşlemleri
    # Tamamen boş sütunları at
    df = df.dropna(axis=1, how='all')
    
    # İlk 8 sütunu al (x1..x7 ve Remarks)
    if len(df.columns) > 8:
        df = df.iloc[:, :8]
        
    # Sütun isimlerini standartlaştır
    df.columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'Remarks']
    
    # 'Remarks' sütununu temizle (Örn: 'Class 1' -> 1)
    df['Remarks'] = df['Remarks'].astype(str).str.replace('Class', '').str.strip()
    # Sadece sayısal olanları tut
    df = df[df['Remarks'].apply(lambda x: x.isdigit())]
    df['Remarks'] = df['Remarks'].astype(int)

    # 3. Örnekleme Fonksiyonu (Stratified Sampling)
    def get_balanced_sample(dataframe, n_per_class, seed):
        # Her sınıftan (1,2,3,4,5) eşit sayıda rastgele örnek al
        return dataframe.groupby('Remarks', group_keys=False).apply(
            lambda x: x.sample(n=min(len(x), n_per_class), random_state=seed)
        )

    # 4. İki Farklı Veri Seti Oluştur (PDF İsteri: 2 Iteration)
    # Set 1 (Seed=42)
    fis_subset_1 = get_balanced_sample(df, n_per_class=20, seed=42)
    
    # Set 2 (Seed=99 - Farklı rastgelelik)
    fis_subset_2 = get_balanced_sample(df, n_per_class=20, seed=99)

    # 5. Dosyaları Kaydet
    fis_subset_1.to_csv('fis_subset_1.csv', index=False)
    fis_subset_2.to_csv('fis_subset_2.csv', index=False)
    
    print(f"✅ Başarılı! Dosyalar oluşturuldu:")
    print(f"   - fis_subset_1.csv ({len(fis_subset_1)} satır)")
    print(f"   - fis_subset_2.csv ({len(fis_subset_2)} satır)")

In [3]:
# Fonksiyonu çalıştır
if __name__ == "__main__":
    create_fis_datasets()

⚙️ FIS İçin Veri Setleri Oluşturuluyor (Her sınıftan 20 örnek)...
✅ Başarılı! Dosyalar oluşturuldu:
   - fis_subset_1.csv (100 satır)
   - fis_subset_2.csv (100 satır)


C:\Users\PC\AppData\Local\Temp\ipykernel_9844\1262913204.py:35: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return dataframe.groupby('Remarks', group_keys=False).apply(
C:\Users\PC\AppData\Local\Temp\ipykernel_9844\1262913204.py:35: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return dataframe.groupby('Remarks', group_keys=False).apply(
